In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from functools import partial
from pathlib import Path

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from einops import rearrange
from scipy import ndimage

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)
    
from pipeline.utils import index_unsorted, nested_insert, nested_select
from pipeline.compact_json_encoder import CompactJSONEncoder

C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
midas_type = "DPT_Large" # "DPT_Large" or "DPT_Hybrid"
model = torch.hub.load("intel-isl/MiDaS", midas_type)
device = torch.device('cuda')
model.to(device)
model.eval()
transform = torch.hub.load('intel-isl/MiDaS', 'transforms').dpt_transform

Using cache found in C:\Users\Cefir/.cache\torch\hub\intel-isl_MiDaS_master
Using cache found in C:\Users\Cefir/.cache\torch\hub\intel-isl_MiDaS_master


In [4]:
dataset_path = Path('D:\\Datasets\\NSD\\')
stimulu_path = dataset_path / 'nsddata_stimuli' / 'stimuli' / 'nsd' / 'nsd_stimuli.hdf5'
embeddings_path = dataset_path / 'derivatives' / 'stimulus_embeddings'
stimulus_images = h5py.File(stimulu_path, 'r')['imgBrick']

In [6]:
N = stimulus_images.shape[0]
@interact(i=(0, N-1))
def show_depth(i):
    image = stimulus_images[i]
    x = transform(image)
    print(x.mean(), x.std(), x.max(), x.min(), x.shape)
    with torch.no_grad():
        depth = model(x.to(device)).cpu()
        depth = F.interpolate(depth[:, None], size=image.shape[:2], mode='bicubic', align_corners=False)
        depth = depth[0, 0].numpy()
        
    figure = plt.figure(figsize=(16, 16))
    grid = figure.add_gridspec(1, 2)
    
    image_ax = figure.add_subplot(grid[0, 0])
    image_ax.axis('off')
    
    depth_ax = figure.add_subplot(grid[0, 1])
    depth_ax.axis('off')

    image_ax.imshow(image)
    depth_ax.imshow(depth)
    plt.show()

interactive(children=(IntSlider(value=36499, description='i', max=72999), Output()), _dom_classes=('widget-int…

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [5]:
embeddings_path = dataset_path / 'derivatives' / 'stimulus_embeddings'

with h5py.File(embeddings_path / f'{midas_type}-depth.hdf5', 'a') as f:
    N = stimulus_images.shape[0]
    dset = f.require_dataset('depth', (N, 384, 384), dtype='float32')
    for i in tqdm(range(0, 54001)):
        x = transform(stimulus_images[i]) 
        with torch.no_grad():
            depth = model(x.to(device)).cpu()
        dset[i] = depth[0]

  0%|          | 0/54001 [00:00<?, ?it/s]

C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\torch\nn\functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
depth_cache = h5py.File(embeddings_path / f'{midas_type}.hdf5', 'r')['depth-pyramid-24'][:]

depth_mean = depth_cache.mean(axis=0)
depth_std = depth_cache.std(axis=0)


N, H, W = depth_cache.shape
@interact(i=(0, N-1))
def show_depth_fft(i):
    depth = depth_cache[i]
    #depth_freq = np.fft.fft2(depth)
    plt.imshow(depth)

interactive(children=(IntSlider(value=36499, description='i', max=72999), Output()), _dom_classes=('widget-int…

In [ ]:
import scipy as sp

depth_cache = h5py.File(embeddings_path / f'{midas_type}-depth.hdf5', 'r')['depth']
N, H, W = depth_cache.shape
@interact(i=(0, N-1), S=(16, H, 16))
def show_depth_downsample(i, S):
    depth = depth_cache[i]
    depth_down = sp.ndimage.zoom(depth, S / H, mode='nearest')
    plt.figure(figsize=(8, 8))
    plt.imshow(depth_down)
    #print(depth.shape)
    #print(depth_down.shape, S/H)

In [ ]:
import scipy as sp
import cv2 as cv

depth_cache = h5py.File(embeddings_path / f'{midas_type}.hdf5', 'r')['depth']
N, H, W = depth_cache.shape
@interact(i=(0, N-1), level=(0, 5))
def show_depth_downsample(i, level):
    depth = depth_cache[i]
    depth_down = depth
    for i in range(level):
        depth_down = cv.pyrDown(depth_down)
    print(depth_down.shape)
    plt.figure(figsize=(8, 8))
    plt.imshow( depth_down)
    plt.colorbar()
    #print(depth.shape)
    #print(depth_down.shape, S/H)

In [11]:
import cv2 as cv

embeddings_path = dataset_path / 'derivatives' / 'stimulus_embeddings'

with h5py.File(embeddings_path / f'{midas_type}.hdf5', 'a') as f:
    depth_dset = f['depth']
    N, H, W = f['depth'].shape
    pyramid_dset = f.require_dataset('depth-pyramid-24', (N, 24, 24), dtype='float32')
    for i in tqdm(range(N)):
        depth_down = f['depth'][i]
        for j in range(4):
            depth_down = cv.pyrDown(depth_down)
        f['depth-pyramid-24'][i] = depth_down

  0%|          | 0/73000 [00:00<?, ?it/s]

In [21]:
with h5py.File(embeddings_path / f'{midas_type}.hdf5', 'r') as f:
    depth = f['depth-pyramid-24'][:]
    
    depth_mean = depth_cache.mean(axis=0, keepdims=True)
    depth_std = depth_cache.std(axis=0, keepdims=True)
    depth = (depth - depth_mean) / depth_std
    N, H, W = depth.shape
    @interact(h=(0, H-1), w=(0, W-1))
    def show_hist(h, w):
        d = depth[:, h, w]
        print(d.min(), d.max(), d.mean(), d.std())
        plt.hist(d, log=True, bins=100)

interactive(children=(IntSlider(value=11, description='h', max=23), IntSlider(value=11, description='w', max=2…

In [12]:
# Look at results of ridge regression depth encoding model

dataset_path = Path('D:\\Datasets\\NSD\\')
derivatives_path = dataset_path / 'derivatives'

encoder_params = h5py.File(derivatives_path / 'fracridge-parameters.hdf5', 'r')
depth_encoder = encoder_params['subj01/DPT_Large/depth-pyramid-24']
sorted_indices = depth_encoder['sorted_indices'][:]
coefs = depth_encoder['coefs']
N = sorted_indices.shape[0]

@interact(n=(0, 100))
def show_coefs(n):
    i, j, k = sorted_indices[n]
    depth_r2 = depth_encoder['r2'][i, j, k]
    print(depth_r2)
    depth_coefs = coefs[i, j, k].reshape(24, 24)
    plt.imshow(depth_coefs)


interactive(children=(IntSlider(value=50, description='n'), Output()), _dom_classes=('widget-interact',))